In [1]:
import sys
import random
import pandas as pd 
import numpy as np
from collections import defaultdict
# third party programs
sys.path.append("../")
import simple_path
from simple_path import StringDB, parse_input



# python stats 



In [2]:
db = StringDB("../Data/STRING.txt")
StringDB_df = db.to_pandas()
fa_input = parse_input("../input/Input.gmt.txt")

In [87]:
# load in the the locus and build a function of 5000 random sub networks within the locus and def generate_random_network(n_networks: int) -> list:
def generate_random_subnetworks(fa_input, gene_counts):
	""" Generates random subnetworks where each gene is selected per locus_name

	Arguments:
	---------
	 fa_input : dict
		loaded FA loci data

	Results:
	-------
	list 
		Randomly selected genes
  
	limitation
	----------
	algortihm is O(N^2) when generating. Therefore increasing the number of generated
	random sublist will make it larger
	""" 
	networks = []
	for gene_list in fa_input.values():
		try:
			gene = gene_counts[random.choice(gene_list)]
		except KeyError:
			gene = 0
		networks.append(gene)
	return networks


def get_connected_genes(string_df):
	""" 
	["description later"]
	"""	
	# generating a set of all known genes within string database
	# -- both columns genes together and converts it into a set (removes all repeats)

	# grouping data based on gene name	
	grouped_df = string_df.groupby(by=["gene1"])
	connected_genes = defaultdict(None)
	for gene, gene_df in grouped_df:
		conn_genes_arr = gene_df["gene2"].values.tolist()
		connected_genes[gene] = conn_genes_arr

	return connected_genes


def get_gene_counts(connected_genes_data):
	"""
	
	"""
	gene_counts = defaultdict(None)
	for gene, connected_genes in connected_genes_data.items():
		gene_counts[gene] = len(connected_genes)

	
	sorted_gene_counts = {k: v for k, v in sorted(gene_counts.items(), key=lambda item: item[1])}

	return sorted_gene_counts	


def label_genes(counts):
    """ labels genes with a unique number """
    labeled_genes = defaultdict(lambda: None)
    for idx, gene_name in enumerate(counts.keys()):
        labeled_genes[idx] = gene_name
    return labeled_genes

def bin_data(labeled_gene_counts, gene_ids, bins=128):
	gene_count_data = tuple(labeled_gene_counts.items())

	# bining data based edge density range using pandas.cut() function
	df = pd.DataFrame(data=gene_count_data, columns=["gene", "counts"])
	edges = np.linspace(df["counts"].values.min(), df["counts"].values.max(), bins+1).astype(int)
	labels = [f'({edges[i]}, {edges[i+1]})' for i in range(len(edges)-1)]
	z = pd.cut(df["counts"], include_lowest=True, bins=bins, labels=labels).to_frame(name="count_range")
	binned_data = z.groupby(by=["count_range"])


	# creating of for loop for adding gene name, edge_density and bin id
	dfs = []
	for bin_id, (name, bin_df) in enumerate(binned_data):
		end = bin_df.index.tolist()
		if len(end) == 0:
			# skip any bins that does not have any data
			continue 
		bin_df["gene_name"] = [gene_ids[idx] for idx in bin_df.index]
		bin_df["edge_density"] = [labeled_gene_counts[gene_ids[idx]] for idx in bin_df.index]
		bin_df["bin"] = bin_id + 1
		dfs.append(bin_df)
	
	binned_df = pd.concat(dfs)
	return binned_df


def permutation_test(ds1, ds2, n_shuffles=1000):
    """ Motivations: Since selections is random and randomly selected genes is selected via bins, we can assume there is exchangeabiltiy
    ds1 : list, np.ndarray
		values of the first data set

	ds2 : list, np.ndarray
		values of the second dataset

  
    Limitations:
    ------------
    >>> Data must be exchangable, it not, the p-values will not be robust 
	>>> Increasing the number of shuffles will slow down the calculations. Has time cimplexity of O(N)
    """
    n_samples, k = len(ds1), 0
    mean_diff = np.abs(np.mean(ds1) - np.mean(ds2))
    merged_ds = np.concatenate([ds1, ds2])
    
    for j in range(n_shuffles):
        np.random.shuffle(merged_ds)
        # conditional of the hpyothesis testing against the mean
        k += mean_diff <= np.abs(np.mean(merged_ds[:n_samples]) - np.mean(merged_ds[n_samples:]))
    return k / n_shuffles

    

## Testing binning

In [90]:
def constuct_cofunctional_network(binned_data, density_array):
	""" Uses edge density value to find which bin the gene is from. Then selects a random gene and extracts its 
 	edge density

	Returns
	-------
	list 
		list containing the edge desnity of the cofunctional_network from genes 
		selected from the same bin. Therefore, in terms of edge density, the random
		subnetowkr and cofunctional_network should be similar.
	"""

	cofunctional_network = []
	for edge_density in density_array:
		if edge_density != 0:
			# -- searches which bin the edge desnity value resides and returns the bin id
			bin_id = binned_data.loc[binned_df["edge_density"] == edge_density]["bin"].values[0]  

			# use the extracted bin id and extract all the edge_densities in the bin and select a random one
			random_edge_density = random.choice(binned_data.loc[binned_df["bin"] == bin_id]["edge_density"].values.tolist())
		else:
			random_edge_density = 0

		cofunctional_network.append(random_edge_density)

	return cofunctional_network


		
def sample_subnetworks(fa_data, gene_counts, binned_data, iterations=100, n_shuffles=100) -> float:
	""" Creates random subnetwork 
 
	Arguments
	---------
 
 
	Results 
	------- 
	float 
		p-value of the simultion 
 	""" 
	if not isinstance(fa_data, dict):
		raise ValueError("Incorrect format provided, requires dictioanry, you have provided {}".format(type(fa_data)))

	cofunc_sum = []
	random_sum = []
	for iteration in range(iterations):
		random_sub_network = generate_random_subnetworks(fa_data, gene_counts)
		cofunctional_network = constuct_cofunctional_network(binned_data, random_sub_network)

		# checking cofunctional and random subnetworks are same size
		if len(random_sub_network) != len(cofunctional_network):
			raise RuntimeError("Both random and cofuctional networks are not the same size")

		# adding up edge desnity score and adding it 
		random_edge_sum = sum(random_sub_network)
		cofunc_edge_sum = sum(cofunctional_network)

		random_sum.append(random_edge_sum)
		cofunc_sum.append(cofunc_edge_sum)


	# permitations test 
	cofunc_arr = np.array(cofunc_sum)
	random_arr = np.array(random_sum)
	p = permutation_test(random_arr, cofunc_arr, n_shuffles=n_shuffles)
	return p


In [91]:

connected_genes = get_connected_genes(StringDB_df)
gene_counts = get_gene_counts(connected_genes)
id_gene = label_genes(gene_counts)



# bin the data 
binned_df = bin_data(gene_counts, id_gene)


# sampling
p_val  = sample_subnetworks(fa_input, gene_counts, binned_df, iterations=1000)

In [92]:
p_val

0.79